## 结构化输出 (Structured Output)

在这个notebook中，我们将演示如何让LLM返回结构化的数据格式，而不是纯文本。我们将使用与chatbot相同的例子：求Normal分布的最大似然估计(MLE)。

### 为什么需要结构化输出？
- 便于程序处理和解析
- 减少格式错误
- 提高数据的可靠性和一致性
- 便于与其他系统集成

In [ ]:
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.messages import HumanMessage, AIMessage, SystemMessage, BaseMessage
import dotenv

dotenv.load_dotenv()

model = ChatGoogleGenerativeAI(model="gemini-2.5-flash")

In [ ]:
import json
from langchain_core.messages import HumanMessage, SystemMessage

# 方法一：在system prompt中规定JSON格式
system_prompt = """
你是一个数学助手。请用以下JSON格式回答问题：
{
  "answer": "详细的数学推导过程",
  "explanation": "简要说明结果的含义"
}

请确保返回的是有效的JSON格式，不要包含任何额外的文本。
"""

user_question = "请推导Normal分布的最大似然估计(MLE)公式"

messages = [
    SystemMessage(content=system_prompt),
    HumanMessage(content=user_question)
]

response = model.invoke(messages)
print("原始回复:")
print(response.content)
print("\n" + "="*50 + "\n")

# 手动解析JSON
try:
    # 尝试解析JSON
    content = str(response.content)
    parsed_result = json.loads(content)
    print("解析成功！")
    print("答案:")
    print(parsed_result["answer"])
    print("\n说明:")
    print(parsed_result["explanation"])
except json.JSONDecodeError as e:
    print(f"JSON解析失败: {e}")
    print("原始内容:")
    print(response.content)


In [ ]:
from pydantic import BaseModel, Field
from typing import Optional

# 定义Pydantic模型
class MathResponse(BaseModel):
    answer: str = Field(description="详细的数学推导过程")
    explanation: str = Field(description="简要说明结果的含义")

# 使用structured output功能
structured_model = model.with_structured_output(MathResponse)

# 直接调用，无需特殊的system prompt
user_question = "请推导Normal分布的最大似然估计(MLE)公式"

print("使用Pydantic结构化输出:")
result = structured_model.invoke(user_question)

print("类型:", type(result))
print("答案:")
print(result.answer)  # type: ignore
print("\n说明:")
print(result.explanation)  # type: ignore

# 可以直接访问属性，也可以转换为字典
print("\n转换为字典:")
if hasattr(result, 'model_dump'):
    result_dict = result.model_dump()  # type: ignore
else:
    result_dict = result
print(result_dict)
